In [1]:
import pickle
from pydantic import BaseModel, Field
from typing import Literal, Optional, List
from openai import OpenAI
import logging
import asyncio
import os
from dotenv import load_dotenv
from pathlib import Path
load_dotenv(Path("../../.env"))

True

In [18]:
from typing import Any, Dict, List, Optional, Literal, ForwardRef

class ToolArguments(BaseModel):
    keys: List[str] = Field(description="A list of arguments to a tool")
    values: List[str] = Field(description="A list of argument values to a tool")
    # arguments: Dict[str, Any] = Field(description="A dictionary where keys are tool arguments and values are the tool call values")


class ToolCall(BaseModel):
    """Represents a tool call request from the LLM."""

    id: str = Field(description="The ID of the tool call.")
    name: str = Field(description="The name of the tool to call.")
    arguments: ToolArguments = Field(description="The arguments to call the tool with.")


class ToolCalls(BaseModel):
    id: int = Field(description="An ID for the tool calls")
    tool_calls: List[ToolCall] = Field(
        description="A list of tools to be executed sequentially."
    )

class PlannerTask(BaseModel):
    """Represents a single task generated by the Planner."""

    id: int = Field(description="Sequential ID for the task.")
    description: str = Field(
        description="Clear description of the task to be executed."
    )
    tool_calls: List[ToolCall] = Field(
        description="A list of tools to be executed sequentially to complete the task"
    )
    thought: str = Field(
        description="A explanation of what needs to be done and how. Includes description and tool calls."
    )
    status: Optional[
        Literal[
            "input_required",
            "completed",
            "error",
            "pending",
            "incomplete",
            "todo",
            "not_started",
        ]
    ] = Field(default="input_required", description="Status of the task")


class Plan(BaseModel):
    """Output schema for the Planner Agent."""

    original_query: str = Field(description="The original user query for context.")
    description: str = Field(description="Clear description of the overall plan.")
    tasks: List[PlannerTask] = Field(
        description="A list of tasks to be executed sequentially."
    )


class ToolResult(BaseModel):
    """Represents the result of a tool execution."""

    tool_call_id: str = Field(description="The ID of the tool call this result is for.")
    result: str = Field(description="The result of the tool execution.")
    is_error: bool = Field(
        default=False, description="Whether the tool execution resulted in an error."
    )


class TaskExecutionResponse(BaseModel):
    """Represents a single task generated by the Planner."""

    id: int = Field(description="Id of task we are executing.")
    description: str = Field(description="Clear description of the task to be executed.")
    tools_sueggested: str = Field(description="A list of the tools suggested for the task")
    response_type: Optional[
        Literal[
            "tool_calls",
            "text",
        ]
    ] = Field(default="input_required", description="The response type of the task execution")
    tool_calls: List[ToolCall] = Field(description="A list of tool calls to be executed. Empty if response_type is text")

In [3]:
from typing import Any, Dict, Optional, Union
from fastmcp.client.client import Client
from contextlib import asynccontextmanager


class MCPClient:
    def __init__(self, config: Union[str, dict] = "http://localhost:8050/sse"):
        """Initialize the MCP client.

        Args:
            config (Union[str, dict]): Either a URL string or a configuration dictionary.
                If string: Treated as the URL of the MCP server.
                If dict: Should follow the MCP configuration format with 'mcpServers' key.
        """
        self.config = config
        self._client = None
        self._is_connected = False

    async def connect(self):
        """Connect to the MCP server(s)."""
        if self._is_connected:
            return

        if isinstance(self.config, str):
            # For SSE transport, we just need the URL
            self._client = Client(self.config)
        else:
            # Configuration mode with multiple servers
            self._client = Client(self.config)

        await self._client.__aenter__()
        self._is_connected = True

    async def disconnect(self):
        """Disconnect from the MCP server(s)."""
        if self._is_connected and self._client:
            await self._client.__aexit__(None, None, None)
            self._is_connected = False
            self._client = None

    @asynccontextmanager
    async def session(self):
        """Context manager for session management."""
        try:
            await self.connect()
            yield self
        finally:
            await self.disconnect()

    async def list_servers(self) -> list:
        """List available MCP servers."""
        if not self._is_connected:
            raise RuntimeError("Not connected to MCP server(s)")
        return list(self._client.servers.keys())

    async def list_tools(self) -> list:
        """List available tools.

        Returns:
            list: List of available tools.
        """
        if not self._is_connected:
            raise RuntimeError("Not connected to MCP server(s)")
        return await self._client.list_tools()

    async def get_tools(self) -> list[dict[str, Any]]:
        """Retrieve tools in a format compatible with OpenAI function calling.

        Returns:
            list[dict[str, Any]]: List of tools in OpenAI function calling format.
        """
        if not self._is_connected:
            raise RuntimeError("Not connected to MCP server(s)")

        tools = await self.list_tools()
        openai_tools = []

        for tool in tools:
            openai_tools.append(
                {
                    "type": "function",
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": {
                        "type": "object",
                        "properties": tool.inputSchema.get("properties", {}),
                        "required": tool.inputSchema.get("required", []),
                    },
                }
            )

        return openai_tools

    async def call_tool(
        self, tool_name: str, arguments: Dict[str, Any], server: Optional[str] = None
    ) -> Any:
        """Call a tool.

        Args:
            tool_name (str): The name of the tool to call.
            arguments (Dict[str, Any]): The arguments to pass to the tool.
            server (str, optional): Specific server to call the tool on.
                                 If None, the client will try to find the tool
                                 in any of the available servers.

        Returns:
            Any: The result of the tool call.
        """
        if not self._is_connected:
            raise RuntimeError("Not connected to MCP server(s)")

        result = await self._client.call_tool(tool_name, arguments, server)
        return result.content[0].text if result.content else None

In [4]:
mcp_client = MCPClient()
await mcp_client.connect()
tools = await mcp_client.get_tools()

In [5]:
tools

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get current temperature for provided coordinates in celsius',
  'parameters': {'type': 'object',
   'properties': {'latitude': {'title': 'latitude', 'type': 'string'},
    'longitude': {'title': 'longitude', 'type': 'string'}},
   'required': ['latitude', 'longitude']}},
 {'type': 'function',
  'name': 'wiki_search',
  'description': 'Search wikipedia for the given query and returns a summary.',
  'parameters': {'type': 'object',
   'properties': {'query': {'title': 'Query', 'type': 'string'},
    'sentences': {'default': 2, 'title': 'Sentences', 'type': 'integer'}},
   'required': ['query']}},
 {'type': 'function',
  'name': 'save_txt',
  'description': 'Save text to a .txt file',
  'parameters': {'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'filename': {'default': 'output.txt',
     'title': 'Filename',
     'type': 'string'}},
   'required': ['text']}},
 {'type': 'function',
  'n

In [20]:
with open("aug_1_plan.pkl", "rb") as f:
    plan_data = pickle.load(f)

In [21]:
plan = Plan(**plan_data)

In [22]:
plan

Plan(original_query='write an essay on the cultural impact of the internet', description='Plan to write an essay on the cultural impact of the internet by researching the topic, selecting main points, writing a structured essay with introduction, body paragraphs, conclusion, followed by review, editing, proofreading, and saving the final essay.', tasks=[PlannerTask(id=1, description="Research the topic 'Cultural impact of the internet' to gather key points and relevant information.", tool_calls=[ToolCall(id='1', name='functions.wiki_search', arguments=ToolArguments(keys=['query', 'sentences'], values=['Cultural impact of the internet', '5']))], thought='I will research the cultural impact of the internet to collect reliable and concise information to use as the basis for the essay.', status='todo'), PlannerTask(id=2, description='Select main points from the research to form the essay structure including key aspects of cultural impact such as communication, social behavior, information 

In [44]:
class Executor:
    """Executor Class.

    Methods:


    Attributes:


    """

    def __init__(self):
        """
        Initialize the orchestrator
        """
        self.tool_call_history: list = []
        self.previous_task_results: list = [{'first task, no previous task yet'}]

    def print_task(self, task: PlannerTask) -> None:
        """Print the given task generated by the planner agent

        Args:
            task: The task to print.
            previous_task_results: The results of the previous task.

        Returns:
            None
        """
        print(f"""
              Executing task: \n
              Task ID: {task.id} \n
              Task description: {task.description} \n
              Task: {task.thought} \n
              Tool calls: {task.tool_calls} \n
              Task status: {task.status} \n
              Previous Task Results: {self.previous_task_results}
        """)

    def print_plan(self, plan: Plan) -> None:
        """Print the given plan generated by the planner agent

        Args:
            plan: The plan to print.

        Returns:
            None
        """
        print(f"""
              Plan to execute is: \n
              Plan ID: {plan.original_query}
              Plan Description: {plan.description}
         """)

    async def call_tools(self, tool_calls: list[dict]) -> list[dict]:
        """Receives a list of tool calls and calls the tools

        Args:
            tool_calls: Either a list of tool call dicts or a string error message

        Returns:
            list[dict]: The results of the tool calls or error information
        """
        # If we received an error message instead of tool calls
        if isinstance(tool_calls, str):
            return [{"error": True, "message": tool_calls}]

        # # Ensure tool_calls is a list
        if not isinstance(tool_calls, list):
            return [
                {
                    "error": True,
                    "message": f"Expected list of tool calls, got {type(tool_calls).__name__}",
                }
            ]
        results = []  # Tool call results
        print('CALLING_TOOLS')
        for tool in tool_calls:  # For each tool
            try:  # Try to call the tool
                if not isinstance(tool, dict):  # If tool is not a dict return error
                    results.append(
                        {
                            "error": True,
                            "message": f"Expected dict, got {type(tool).__name__}",
                        }
                    )
                    continue
                # Extract tool name and arguments
                name = tool['name']
                arguments = tool['arguments']
                print(f'CALLING TOOL: {name}')
                if not name:
                    results.append(
                        {"error": True, "message": "Tool call missing 'name' field"}
                    )
                    continue

                # Call the tool through MCP client
                result = await mcp_client.call_tool(name, arguments)
                # append tool call reults. Includes name, arguments, and result
                results.append(
                    {"result": result}
                )
                self.tool_call_history.append({"name": name, "arguments": arguments, "result": result, "error": False})
            
            # Handle exceptions
            except Exception as e:
                print("AT EXCEPTION")
                results.append(
                    {
                        "error": True,
                        "name": name if "name" in locals() else "unknown",
                        "message": f"Error calling tool: {str(e)}",
                    }
                )
        print(f'TOOL CALL RESULTS: {results}')
        return results
  
    async def execute_task(self, task: PlannerTask) -> list[dict]:
        """Execute the given task generated by the planner agent

        Args:
            task: The task to execute.

        Returns:
            A list of results

        """

        # print current task
        self.print_task(task)
        tool_calls = [] # list to hold tool_calls in current task
        for i in range(len(task.tool_calls)): # for every tool call in the task
            tool_call = task.tool_calls[i] # select the tool call
            tools = self.extract_tools(tool_call) # extract the tool into {name: tool_name, arguments: {...} 
            tool_calls.append(tools) # add tool to tool_Calls list

        print(f'TOOL_CALLS: {tool_calls}')
        # tool_call_results = await self.call_tools(tool_calls) # call the tools, should return [{'result': result} ...]
        # results = [result["result"] for result in tool_call_results if "result" in result] # get the results only
        
        # return results
        # return ''

    def extract_tools(self, tool_call):
    """Extract tool name and arguments from a tool_call object."""
        name = tool_call.name.split('.')[-1]
        tool = {
            "name": name,
            "arguments": {}
        }
    
        keys = tool_call.arguments.keys
        values = tool_call.arguments.values
    
        # Sanity check
        if len(keys) != len(values):
            raise ValueError(f"Tool call argument mismatch: keys={keys}, values={values}")
    
        # Handle each tool separately
        for i in range(len(keys)):
            key = keys[i]
            value = values[i]
    
            # Writer tool
            if name == "writer_tool":
                    if key == "content":
                    # Replace model-generated content with previous task results
                    tool["arguments"]["context"] = str(self.tool_call_history)
                elif key == "query":
                    tool["arguments"]["query"] = value
                else:
                    tool["arguments"][key] = value
    
            # Review tool or assembler — inject full previous results
            elif name in ("review_tool", "assemble_content"):
                if key == "content":
                    tool["arguments"]["content"] = str(self.previous_task_results)
                else:
                    tool["arguments"][key] = value
    
            # Save tool — replace content if necessary
            elif name == "save_txt":
                if key == "text":
                    tool["arguments"]["text"] = str(self.previous_task_results)
                elif key == "filename":
                    tool["arguments"]["filename"] = value
                else:
                    tool["arguments"][key] = value
    
            # Default case — generic fallback
            else:
                tool["arguments"][key] = value

        return tool


    
    async def execute_plan(self, plan: Plan) -> list:
        """Execute the given plan generated by the planner agent

        Args:
            plan: The plan to execute.

        Returns:
            A list of results
        """
        self.print_plan(plan) # print the plan
        results = [
            {
                'task': 'No task yet',
                'results': 'No task results yet',
            }
        ] # list to hold results of each task execution.
        for i in range(len(plan.tasks)): # iterate through tasks
            task: PlannerTask = plan.tasks[i] # select the task
            res = await self.execute_task(task) # execute task
            # append task execution results to list
            self.previous_task_results.append(
                    {
                        'task_id': task.id,
                        'task': task.description,
                        'results': res,
                    }
            )

        return results

In [47]:
executor = Executor()

In [48]:
plan.tasks[0]

PlannerTask(id=1, description="Research the topic 'Cultural impact of the internet' to gather key points and relevant information.", tool_calls=[ToolCall(id='1', name='functions.wiki_search', arguments=ToolArguments(keys=['query', 'sentences'], values=['Cultural impact of the internet', '5']))], thought='I will research the cultural impact of the internet to collect reliable and concise information to use as the basis for the essay.', status='todo')

In [49]:
res = await executor.execute_plan(plan)


              Plan to execute is: 

              Plan ID: write an essay on the cultural impact of the internet
              Plan Description: Plan to write an essay on the cultural impact of the internet by researching the topic, selecting main points, writing a structured essay with introduction, body paragraphs, conclusion, followed by review, editing, proofreading, and saving the final essay.
         

              Executing task: 

              Task ID: 1 

              Task description: Research the topic 'Cultural impact of the internet' to gather key points and relevant information. 

              Task: I will research the cultural impact of the internet to collect reliable and concise information to use as the basis for the essay. 

              Tool calls: [ToolCall(id='1', name='functions.wiki_search', arguments=ToolArguments(keys=['query', 'sentences'], values=['Cultural impact of the internet', '5']))] 

              Task status: todo 

              Previous Tas

In [25]:
res

[{'task': 'No task yet', 'results': 'No task results yet'}]

In [27]:
dir(executor)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'call_tools',
 'execute_plan',
 'execute_task',
 'extract_tools',
 'previous_task_results',
 'print_plan',
 'print_task',
 'tool_call_history']

In [28]:
executor.previous_task_results

[{'first task, no previous task yet'},
 {'task_id': 1,
  'task': "Research the topic 'Cultural impact of the internet' to gather key points and relevant information.",
  'results': ['The American singer-songwriter Beyoncé has had a significant cultural impact through her music, visuals, performances, image, politics and lifestyle. She has received widespread acclaim and numerous accolades throughout her career, solidifying her position as an influential cultural icon and one of the greatest artists of all time according to numerous major publications.\nBeyoncé has revolutionized the music industry, transforming the production, distribution, promotion, and consumption of music. She has been credited with reviving both the album and the music video as art forms, popularizing surprise albums and visual albums, and changing the Global Release Day to Friday. Her artistic innovations, such as staccato rap-singing and chopped and re-pitched vocals, have become defining features of 21st centur

In [29]:
executor.tool_call_history

[{'name': 'wiki_search',
  'arguments': {'query': 'Cultural impact of the internet', 'sentences': '5'},
  'result': 'The American singer-songwriter Beyoncé has had a significant cultural impact through her music, visuals, performances, image, politics and lifestyle. She has received widespread acclaim and numerous accolades throughout her career, solidifying her position as an influential cultural icon and one of the greatest artists of all time according to numerous major publications.\nBeyoncé has revolutionized the music industry, transforming the production, distribution, promotion, and consumption of music. She has been credited with reviving both the album and the music video as art forms, popularizing surprise albums and visual albums, and changing the Global Release Day to Friday. Her artistic innovations, such as staccato rap-singing and chopped and re-pitched vocals, have become defining features of 21st century popular music.',
  'error': False}]